In [16]:
import pandas as pd
import sql_functions as sf
import psycopg2
import numpy as np

In [2]:
schema = 'capstone_travel_index'

In [5]:
city = sf.get_dataframe(f'SELECT * FROM {schema}.country_city_code')

In [6]:
city.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,None,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519


In [10]:
city.columns

Index(['city', 'city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3',
       'admin_name', 'capital', 'population', 'id'],
      dtype='object')

In [11]:
city_lat_lud = city.drop(['city', 
                        'country', 
                        'iso2', 
                        'iso3',
                        'admin_name', 
                        'capital', 
                        'population', 
                        'id'], axis=1)

In [12]:
city_lat_lud

,city_ascii,lat,lng
0,Tokyo,35.6850,139.7514
1,New York,40.6943,-73.9249
2,Mexico City,19.4424,-99.1310
3,Mumbai,19.0170,72.8570
4,Sao Paulo,-23.5587,-46.6250
...,...,...,...
15488,Timmiarmiut,62.5333,-42.2167
15489,Cheremoshna,51.3894,30.0989
15490,Ambarchik,69.6510,162.3336
15491,Nordvik,74.0165,111.5100


In [25]:
frankfurt = city_lat_lud[city_lat_lud['city_ascii'] == "'s-Hertogenbosch"]

In [26]:
frankfurt

,city_ascii,lat,lng,distance
2758,'s-Hertogenbosch,51.6833,5.3167,294.026179


In [17]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two sets of latitude and longitude coordinates.
    """
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    
    # Radius of the Earth in kilometers
    radius = 6371
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = radius * c
    
    return distance

In [18]:
# Origin coordinates
origin_lat = city_lat_lud.loc[127, 'lat']
origin_lon = city_lat_lud.loc[127, 'lng']

# Calculate the distance between the origin and all other locations
for index, row in city_lat_lud.iterrows():
    distance = haversine(origin_lat, origin_lon, row['lat'], row['lng'])
    city_lat_lud.at[index, 'distance'] = distance

In [43]:
city_lat_lud[city_lat_lud['distance'] == city_lat_lud['distance'].max()]

,city_ascii,lat,lng,distance
5339,Waitangi,-44.0263,-176.3696,19239.548859


Flugzeug 230 g CO2
Auto (1,5 Personen)	147 g CO2
Zug Fernverkehr	32 g CO2

In [49]:
#Co2 per km source: https://www.mein-klimaschutz.de/unterwegs/a/einkauf/welches-verkehrsmittel-verursacht-im-vergleich-mehr-co2/
co_dic = {'airplane':230, 'car':147, 'train':32}

In [54]:
footprint = []
for index, row in city_lat_lud.iterrows():
    dist = city_lat_lud['distance'][index]
    if dist > 800:
        co = dist * co_dic['airplane']
    elif dist <= 800 and dist > 500:
        co = dist * co_dic['train']
    else:
        co = dist * co_dic['car']
    footprint.append(co)

In [55]:
footprint

[2147336.4109650785,
 1425718.0194060649,
 2198503.533287438,
 1510530.132687284,
 2260552.1864845958,
 1406171.5220343596,
 2028543.276495054,
 1698731.1792071038,
 2140156.0100554493,
 1709779.7874348762,
 2641764.1233690656,
 1306209.7651831093,
 671412.2769894309,
 2205386.17821508,
 2122992.2391149267,
 1789123.521830446,
 2363556.224055573,
 465056.0129922088,
 428339.88645431516,
 70271.3194499174,
 1966956.0056851793,
 1122249.4505992206,
 2556472.4088128903,
 2077593.1616653095,
 1603355.7038052264,
 20375.7121763647,
 2468216.725243449,
 866960.2640849663,
 1399745.057935328,
 2092840.170173125,
 2101171.996142803,
 1943830.080421178,
 2107044.637648127,
 1815061.769627745,
 1744928.251556485,
 2157072.172087642,
 1702938.196472032,
 2062823.8132707414,
 1309310.709337691,
 1852724.442813822,
 1785686.6389282253,
 1632958.1664429319,
 1898169.6913799064,
 2785048.0534874583,
 1455054.0421160795,
 2147488.183959968,
 332380.86771799624,
 1937807.4930095098,
 1431139.6106843199

In [56]:
city_lat_lud['footprint'] = footprint

In [58]:
city_lat_lud[city_lat_lud['city_ascii'] == 'Barcelona']

,city_ascii,lat,lng,distance,footprint
56,Barcelona,41.3833,2.1834,1091.352620,2.510111e+05
916,Barcelona,10.1304,-64.7200,7963.319272,1.831563e+06
